## Connect to mongo

In [3]:
from pymongo import MongoClient

client = MongoClient('127.0.0.1:27017')
db = client.Main

## Tweets from mongo into pandas df


Pandas has an out-of-the-box `json_normalize` function which does a good job of handling everything except nested lists/arrays. Due to the structured nature of tweets I think the easiest way is to just let it do its thing first, then to transform the resulting data frame. You could also write your own JSON parser but my guess is most people would rather do the transforms within pandas. 

In [133]:
import pandas as pd
import json
from pandas.io.json import json_normalize

#Read 10 tweets
_cursor = db.tweets.find().limit( 10 )
tweets = []
while _cursor.alive:
    tweets.append(_cursor.next())
#tweets

In [134]:
#JSON parser - not complete, but I will leave this here in case someone wants to use a SQL database or something

def flatten_docs(tweets):
    for tweet in tweet:
        tweet = flatten_doc(tweet)
    
    return tweets

def flatten_doc(tweet):
    for key in tweet:
        print(type(tweet[key]))
        #if type(tweet[key]) is dict then pass the child to this function
        
#flatten_docs(tweets)

In [135]:
#Pandas - is better 
df = json_normalize(tweets)
#df.dtypes
#df.head()
df

,_id,contributors,coordinates,created_at,display_text_range,entities.hashtags,entities.symbols,entities.urls,entities.user_mentions,extended_tweet.display_text_range,...,user.profile_text_color,user.profile_use_background_image,user.protected,user.screen_name,user.statuses_count,user.time_zone,user.translator_type,user.url,user.utc_offset,user.verified
0,59d030869ded1321639a2fb8,None,None,Sun Oct 01 00:02:14 +0000 2017,NaN,"[{'indices': [18, 28], 'text': 'Livescore'}, {...",[],[],"[{'screen_name': 'ScoresPro', 'name': 'ScoresP...",NaN,...,333333,True,False,ScoresProLive,1571310,Athens,none,http://www.scorespro.com/livescore/,10800.0,False
1,59d030859ded1321639a2fb7,None,None,Sun Oct 01 00:02:13 +0000 2017,NaN,"[{'indices': [38, 44], 'text': 'AF066'}]",[],[],"[{'screen_name': 'SamiSiddiqi1', 'name': 'Sami...",NaN,...,000000,False,False,fly_cow,21818,Tokyo,none,None,32400.0,False
2,59d030889ded1321639a2fb9,None,None,Sun Oct 01 00:02:16 +0000 2017,NaN,"[{'indices': [38, 44], 'text': 'AF066'}]",[],[],"[{'screen_name': 'SamiSiddiqi1', 'name': 'Sami...",NaN,...,0084B4,True,False,omotedoor,4877,Hawaii,none,None,-36000.0,False
3,59d030899ded1321639a2fba,None,None,Sun Oct 01 00:02:16 +0000 2017,NaN,"[{'indices': [57, 64], 'text': 'nbto17'}]",[],[{'display_url': 'abitmoredetail.wordpress.com...,[],NaN,...,333333,True,False,randyfmcdonald,21071,Eastern Time (US & Canada),none,https://about.me/randy_mcdonald/,-14400.0,False
4,59d030899ded1321639a2fbb,None,None,Sun Oct 01 00:02:17 +0000 2017,NaN,[],[],[],"[{'screen_name': 'cnni', 'name': 'CNN Internat...",NaN,...,333333,True,False,hardchichi,1785,None,none,None,NaN,False
5,59d0308a9ded1321639a2fbc,None,None,Sun Oct 01 00:02:18 +0000 2017,NaN,"[{'indices': [82, 90], 'text': 'Toronto'}, {'i...",[],[{'display_url': 'jobfindly.com/entry-level-cu...,[],NaN,...,333333,True,False,jobfindlymar,161612,None,none,https://www.jobfindly.com/marketing-jobs.html,NaN,False
6,59d0308b9ded1321639a2fbd,None,None,Sun Oct 01 00:02:19 +0000 2017,NaN,[],[],[{'display_url': 'twitter.com/cbctoronto/sta…'...,"[{'screen_name': 'desusnice', 'name': 'Desus N...",NaN,...,333333,True,False,wbsmets,8310,Atlantic Time (Canada),none,http://willbuchanan.bandcamp.com,-10800.0,False
7,59d030909ded1321639a2fbe,None,None,Sun Oct 01 00:02:23 +0000 2017,"[0, 140]",[],[],[{'display_url': 'twitter.com/i/web/status/9…'...,"[{'screen_name': 'laminmanneh21', 'name': 'Lam...","[0, 131]",...,333333,True,False,MariamCoco411,3327,None,none,None,NaN,False
8,59d030909ded1321639a2fbf,None,None,Sun Oct 01 00:02:24 +0000 2017,NaN,[],[],[],"[{'screen_name': 'Sweethslw', 'name': 'Fränwiw...",NaN,...,333333,True,False,madeinglorydays,29842,Santiago,none,None,-10800.0,False
9,59d030929ded1321639a2fc0,None,None,Sun Oct 01 00:02:25 +0000 2017,NaN,[],[],[{'display_url': 'twitter.com/TorontoStar/st…'...,"[{'screen_name': 'max_read', 'name': 'Max Read...",NaN,...,333333,True,False,gillmcguane,11568,None,none,https://www.instagram.com/gillmcguane/,NaN,False


In [136]:
s = df.iloc[0:1,:]['entities.hashtags']

In [142]:
def flatten_hashtags(x):
    s = ""
    for k in x:
        s += k['text'] + ','
    return s[:-1]
        
df['hashtags'] = df['entities.hashtags'].apply(lambda x: flatten_hashtags(x) if len(x) else "")
df = df.drop('entities.hashtags', axis=1)


In [149]:
def flatten_urls(x):
    s=""
    for k in x:
        s += k['display_url'] + ','
    return s[:-1]

df['urls'] = df['entities.urls'].apply(lambda x: flatten_urls(x) if len(x) else "")

In [150]:
df['urls']

0                                                
1                                                
2                                                
3    abitmoredetail.wordpress.com/2017/09/30/urb…
4                                                
5                   jobfindly.com/entry-level-cu…
6                     twitter.com/cbctoronto/sta…
7                     twitter.com/i/web/status/9…
8                                                
9                     twitter.com/TorontoStar/st…
Name: urls, dtype: object

I might have missed some columns that need to be flattened. I'm also not sure what the deal is with the URLs getting truncated.